# Flights Data Exploratory Analysis using PySpark – King Khalid International Airport (RUH)

This notebook presents a comprehensive Exploratory Data Analysis (EDA) of flight operations associated with King Khalid International Airport (RUH).
The dataset includes detailed information about airlines, aircraft, flight statuses, terminals, schedules, and destinations.
The objective of this analysis is to understand flight distribution patterns, airline activity, terminal utilization, and time-based trends that characterize RUH’s air traffic.

*italicized text*
These insights lay the foundation for future stages such as flight-delay prediction, demand forecasting, or terminal resource optimization using machine learning or simulation models.

In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("PySpark Data Analysis") \
    .getOrCreate()

In [3]:
df = spark.read.parquet("/content/flights_RUH.parquet")
df.show(5)

+-------------+---------------+------------+--------------+-----------------+------------+------------+-------+-----------+---------------+-------+--------+-------------------+-------------------+-------------------+-----------------+----------------+------------------------+------------------------+------------------------+-------------------------+--------------------------+----------------------------+
|flight_number| aircraft.model|aircraft.reg|aircraft.modeS|     airline.name|airline.iata|airline.icao| status|flight_type|codeshareStatus|isCargo|callSign|origin_airport_name|origin_airport_icao|origin_airport_iata|movement.terminal|movement.quality|destination_airport_icao|destination_airport_iata|destination_airport_name|movement.airport.timeZone|movement.scheduledTime.utc|movement.scheduledTime.local|
+-------------+---------------+------------+--------------+-----------------+------------+------------+-------+-----------+---------------+-------+--------+-------------------+------

In [4]:
from pyspark.sql import functions as F
def q(name: str):
    return F.col(f"`{name}`")

null_counts = df.select([
    F.sum(F.when(q(c).isNull(), 1).otherwise(0)).alias(c)
    for c in df.columns
])
null_counts.show(truncate=False)

+-------------+--------------+------------+--------------+------------+------------+------------+------+-----------+---------------+-------+--------+-------------------+-------------------+-------------------+-----------------+----------------+------------------------+------------------------+------------------------+-------------------------+--------------------------+----------------------------+
|flight_number|aircraft.model|aircraft.reg|aircraft.modeS|airline.name|airline.iata|airline.icao|status|flight_type|codeshareStatus|isCargo|callSign|origin_airport_name|origin_airport_icao|origin_airport_iata|movement.terminal|movement.quality|destination_airport_icao|destination_airport_iata|destination_airport_name|movement.airport.timeZone|movement.scheduledTime.utc|movement.scheduledTime.local|
+-------------+--------------+------------+--------------+------------+------------+------------+------+-----------+---------------+-------+--------+-------------------+-------------------+-------

In [5]:
cols_to_drop = ['aircraft.reg', 'callSign', 'aircraft.modeS']
df = df.drop(*cols_to_drop)

In [6]:
for c in df.columns:
    df = df.withColumnRenamed(c, c.replace('.', '_'))

df.printSchema()

root
 |-- flight_number: string (nullable = true)
 |-- aircraft_model: string (nullable = true)
 |-- airline_name: string (nullable = true)
 |-- airline_iata: string (nullable = true)
 |-- airline_icao: string (nullable = true)
 |-- status: string (nullable = true)
 |-- flight_type: string (nullable = true)
 |-- codeshareStatus: string (nullable = true)
 |-- isCargo: boolean (nullable = true)
 |-- origin_airport_name: string (nullable = true)
 |-- origin_airport_icao: string (nullable = true)
 |-- origin_airport_iata: string (nullable = true)
 |-- movement_terminal: string (nullable = true)
 |-- movement_quality: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- destination_airport_icao: string (nullable = true)
 |-- destination_airport_iata: string (nullable = true)
 |-- destination_airport_name: string (nullable = true)
 |-- movement_airport_timeZone: string (nullable = true)
 |-- movement_scheduledTime_utc: string (nullable = true)
 |-- movement_scheduledT

In [7]:
cols_to_drop = ['aircraft_reg', 'callSign', 'aircraft_modeS']
df = df.drop(*cols_to_drop)

In [8]:
fill_values = {
    'airline_icao': 'Unknown',
    'aircraft_model': 'Unknown',
    'airline_iata': 'Unknown',
    'movement_terminal': 'Unknown',
    'destination_airport_icao': 'Unknown',
    'destination_airport_iata': 'Unknown',
    'movement_airport_timeZone': 'Unknown',
}
df = df.fillna(fill_values)

In [19]:
core_cols = [
    "flight_number", "airline_name", "departure_date",
    "status", "origin_airport_name", "destination_airport_name"
]

clean_df = df.dropDuplicates(core_cols)

In [20]:
clean_df.groupBy("flight_number", "departure_date").count().filter(F.col("count") > 1).count()

557

In [14]:
from pyspark.sql import functions as F
clean_df = df.filter(
    F.lower(F.col("status")).isin(["departed", "canceled"])
)

# total number of flights and distinct airlines

In [21]:
from pyspark.sql import functions as F
from pyspark.sql import Window

df = df.withColumn("departure_date", F.to_date("movement_scheduledTime_utc"))

clean_df = df.filter(
    F.lower(F.col("status")).isin(["departed", "canceled"])
)

window_spec = Window.partitionBy("flight_number", "departure_date") \
                   .orderBy(F.col("movement_scheduledTime_utc").desc())
df_ranked = clean_df.withColumn("rank", F.row_number().over(window_spec))

latest_flights = df_ranked.filter(F.col("rank") == 1).drop("rank")
latest_flights.selectExpr(
    "count(*) as total_real_flights",
    "count(distinct airline_name) as unique_airlines"
).show()

+------------------+---------------+
|total_real_flights|unique_airlines|
+------------------+---------------+
|             11557|             31|
+------------------+---------------+



#  number of airlines serving each destination

In [22]:
clean_df.groupBy("destination_airport_name") \
    .agg(F.countDistinct("airline_name").alias("num_airlines")) \
    .orderBy(F.col("num_airlines").desc()) \
    .show(10, truncate=False)

+------------------------+------------+
|destination_airport_name|num_airlines|
+------------------------+------------+
|Cairo                   |8           |
|Dubai                   |5           |
|Amman                   |4           |
|Istanbul                |4           |
|Doha                    |4           |
|New Delhi               |4           |
|Vienna                  |3           |
|London                  |3           |
|Medina                  |3           |
|Athens                  |2           |
+------------------------+------------+
only showing top 10 rows



# average daily flights per airline

In [24]:
clean_df = clean_df.withColumn("departure_date", F.to_date("movement_scheduledTime_utc"))
avg_daily_flights = (
    clean_df.groupBy("airline_name", "departure_date")
    .count()
    .groupBy("airline_name")
    .agg(F.avg("count").alias("avg_daily_flights"))
    .orderBy(F.col("avg_daily_flights").desc())
)
avg_daily_flights.show(10, truncate=False)

+-------------+------------------+
|airline_name |avg_daily_flights |
+-------------+------------------+
|Saudi Arabian|20.085308056872037|
|flynas       |11.528571428571428|
|flyadeal     |7.304761904761905 |
|Qatar        |4.319047619047619 |
|flydubai     |1.8775510204081634|
|British      |1.8349514563106797|
|Turkish      |1.7391304347826086|
|EgyptAir     |1.4090909090909092|
|Emirates     |1.2272727272727273|
|Qatar Airways|1.2244897959183674|
+-------------+------------------+
only showing top 10 rows



In [25]:
canceled_avg = (
    clean_df.filter(F.lower(F.col("status")) == "canceled")
    .groupBy("airline_name", "departure_date")
    .count()
    .groupBy("airline_name")
    .agg(F.avg("count").alias("avg_daily_canceled"))
    .orderBy(F.col("avg_daily_canceled").desc())
)

canceled_avg.show(10, truncate=False)

+--------------+------------------+
|airline_name  |avg_daily_canceled|
+--------------+------------------+
|Qatar Airways |2.0               |
|Qatar         |2.0               |
|flyadeal      |2.0               |
|LOT - Polish  |2.0               |
|Aegean        |2.0               |
|Saudi Arabian |1.7692307692307692|
|British       |1.6153846153846154|
|Air India     |1.5               |
|Cathay Pacific|1.0               |
|flynas        |1.0               |
+--------------+------------------+
only showing top 10 rows



#  busiest day overall

In [26]:
(df.groupBy("departure_date")
   .agg(F.count("*").alias("num_flights"))
   .orderBy(F.col("num_flights").desc())
   .limit(1)
   .show())

+--------------+-----------+
|departure_date|num_flights|
+--------------+-----------+
|    2025-08-11|        814|
+--------------+-----------+



# busiest destination-airline combinations


In [27]:
clean_df.groupBy("destination_airport_name") \
    .count() \
    .orderBy(F.col("count").desc()) \
    .show(10, truncate=False)

+------------------------+-----+
|destination_airport_name|count|
+------------------------+-----+
|Medina                  |3304 |
|Doha                    |1483 |
|London                  |1215 |
|Dubai                   |1160 |
|Istanbul                |776  |
|New Delhi               |325  |
|Frankfurt-am-Main       |292  |
|Tbilisi                 |221  |
|Manila                  |219  |
|Athens                  |218  |
+------------------------+-----+
only showing top 10 rows



# Top 10 most frequent routes (origin → destination)

In [29]:
clean_df.groupBy("origin_airport_name", "destination_airport_name") \
    .count() \
    .orderBy(F.col("count").desc()) \
    .show(10, truncate=False)

+-------------------+------------------------+-----+
|origin_airport_name|destination_airport_name|count|
+-------------------+------------------------+-----+
|Riyadh             |Medina                  |3304 |
|Riyadh             |Doha                    |1483 |
|Riyadh             |London                  |1215 |
|Riyadh             |Dubai                   |1160 |
|Riyadh             |Istanbul                |776  |
|Riyadh             |New Delhi               |325  |
|Riyadh             |Frankfurt-am-Main       |292  |
|Riyadh             |Tbilisi                 |221  |
|Riyadh             |Manila                  |219  |
|Riyadh             |Athens                  |218  |
+-------------------+------------------------+-----+
only showing top 10 rows



# What are the top 10 airlines by number of flights?

In [30]:
top_airlines = (
    clean_df.groupBy("airline_name")
    .count()
    .orderBy(F.col("count").desc())
)
top_airlines.show(10, truncate=False)

+---------------+-----+
|airline_name   |count|
+---------------+-----+
|Saudi Arabian  |4238 |
|flynas         |2421 |
|flyadeal       |1534 |
|Qatar          |907  |
|British        |378  |
|Turkish        |320  |
|Lufthansa      |204  |
|Virgin Atlantic|191  |
|flydubai       |184  |
|China Eastern  |159  |
+---------------+-----+
only showing top 10 rows



# What are the most common flight statuses?


In [31]:
clean_df.groupBy("status").count().orderBy(F.col("count").desc()).show()

+--------+-----+
|  status|count|
+--------+-----+
|Departed|11518|
|Canceled|   78|
+--------+-----+



# Which are the top 10 destination airports?

In [33]:
clean_df.groupBy("destination_airport_name") \
    .count() \
    .orderBy(F.col("count").desc()) \
    .show(10, truncate=False)

+------------------------+-----+
|destination_airport_name|count|
+------------------------+-----+
|Medina                  |3304 |
|Doha                    |1483 |
|London                  |1215 |
|Dubai                   |1160 |
|Istanbul                |776  |
|New Delhi               |325  |
|Frankfurt-am-Main       |292  |
|Tbilisi                 |221  |
|Manila                  |219  |
|Athens                  |218  |
+------------------------+-----+
only showing top 10 rows



# What’s the daily flight volume trend?

In [34]:
clean_df = clean_df.withColumn("departure_date", F.to_date("movement_scheduledTime_utc"))
daily_flights = (
    clean_df.groupBy("departure_date")
    .agg(F.count("*").alias("num_flights"))
    .orderBy("departure_date")
)

daily_flights.show(10, truncate=False)

+--------------+-----------+
|departure_date|num_flights|
+--------------+-----------+
|2025-03-14    |8          |
|2025-03-15    |47         |
|2025-03-16    |43         |
|2025-03-17    |53         |
|2025-03-18    |41         |
|2025-03-19    |46         |
|2025-03-20    |47         |
|2025-03-21    |50         |
|2025-03-22    |54         |
|2025-03-23    |47         |
+--------------+-----------+
only showing top 10 rows



# Which airlines fly to the most destinations?

In [35]:
clean_df.groupBy("airline_name") \
    .agg(F.countDistinct("destination_airport_name").alias("num_destinations")) \
    .orderBy(F.col("num_destinations").desc()) \
    .show(10, truncate=False)

+----------------+----------------+
|airline_name    |num_destinations|
+----------------+----------------+
|Saudi Arabian   |28              |
|flynas          |19              |
|flyadeal        |7               |
|EgyptAir        |1               |
|Cathay Pacific  |1               |
|Qatar Airways   |1               |
|Turkish         |1               |
|ITA             |1               |
|Air Arabia Egypt|1               |
|Air France      |1               |
+----------------+----------------+
only showing top 10 rows



# How many terminals are used at RUH?

In [37]:
clean_df.filter(F.col("origin_airport_iata") == "RUH") \
    .select("movement_terminal") \
    .distinct() \
    .show()

+-----------------+
|movement_terminal|
+-----------------+
|                3|
|                5|
|          Unknown|
|                1|
|                4|
|                2|
+-----------------+



# Number of Airlines Serving Each Destination

In [38]:
import plotly.express as px
from pyspark.sql import functions as F

dest_airlines = (
    clean_df.groupBy("destination_airport_name")
    .agg(F.countDistinct("airline_name").alias("num_airlines"))
    .orderBy(F.col("num_airlines").desc())
)
dest_airlines_pd = dest_airlines.limit(20).toPandas()
fig = px.bar(
    dest_airlines_pd,
    x="destination_airport_name",
    y="num_airlines",
    color="num_airlines",
    title="Number of Airlines Serving Each Destination (Cleaned Data)",
    color_continuous_scale="Blues",
    text="num_airlines"
)
fig.update_traces(textposition="outside")
fig.update_layout(
    xaxis_title="Destination Airport",
    yaxis_title="Number of Airlines",
    xaxis_tickangle=-45,
    template="plotly_white",
    title_font=dict(size=18, family="Arial", color="black"),
    margin=dict(l=40, r=40, t=60, b=120)
)

fig.show()

# Top 15 Airlines by Number of Flights

In [39]:
airline_counts = (
    clean_df.groupBy("airline_name")
    .count()
    .orderBy(F.col("count").desc())
    .limit(15)
    .toPandas()
)
fig = px.bar(
    airline_counts,
    x="airline_name",
    y="count",
    title="Top 15 Airlines by Number of Flights (Cleaned Data)",
    text="count",
    color="count",
    color_continuous_scale="Blues"
)
fig.update_traces(textposition="outside")
fig.update_layout(
    xaxis_title="Airline",
    yaxis_title="Flight Count",
    xaxis_tickangle=-45,
    template="plotly_white",
    title_font=dict(size=18, family="Arial", color="black"),
    margin=dict(l=40, r=40, t=60, b=100)
)

fig.show()

# Daily Flight Volume Over Time

In [40]:
clean_df = clean_df.withColumn("departure_date", F.to_date("movement_scheduledTime_utc"))
daily_trend = (
    clean_df.groupBy("departure_date")
    .agg(F.count("*").alias("num_flights"))
    .orderBy("departure_date")
    .toPandas()
)
fig = px.line(
    daily_trend,
    x="departure_date",
    y="num_flights",
    title="Daily Flight Volume Over Time (Cleaned Data)",
    markers=True
)
fig.update_layout(
    template="plotly_white",
    xaxis_title="Date",
    yaxis_title="Number of Flights",
    title_font=dict(size=18, family="Arial", color="black"),
    margin=dict(l=40, r=40, t=60, b=80)
)

fig.show()

# Flight Status Distribution by Airline

In [41]:
top10 = (
    clean_df.groupBy("airline_name")
    .count()
    .orderBy(F.col("count").desc())
    .limit(10)
    .select("airline_name")
)
data = (
    clean_df.join(top10, "airline_name")
    .groupBy("airline_name", "status")
    .count()
    .orderBy("airline_name", "status")
    .toPandas()
)
fig = px.bar(
    data,
    x="airline_name",
    y="count",
    color="status",
    title="Flight Status Distribution by Airline (Top 10 - Cleaned Data)",
    text="count",
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_layout(
    xaxis_tickangle=-45,
    template="plotly_white",
    yaxis_title="Number of Flights",
    xaxis_title="Airline",
    legend_title="Flight Status",
    title_font=dict(size=18, family="Arial", color="black"),
    margin=dict(l=40, r=40, t=60, b=100)
)

fig.show()

# Airline Coverage by Destination

In [42]:
cover = (
    clean_df.groupBy("airline_name", "destination_airport_name")
    .agg(F.count("*").alias("flights"))
    .orderBy(F.col("flights").desc())
    .toPandas()
)

fig = px.treemap(
    cover,
    path=["airline_name", "destination_airport_name"],
    values="flights",
    title="Airline Coverage by Destination (Cleaned Data)",
    color="flights",
    color_continuous_scale="Blues"
)

fig.update_layout(
    template="plotly_white",
    title_font=dict(size=18, family="Arial", color="black"),
    margin=dict(l=40, r=40, t=60, b=40)
)

fig.show()

# Top 20 Destinations by Flight Count

In [43]:
dest_counts = (
    clean_df.groupBy("destination_airport_name")
    .agg(F.count("*").alias("num_flights"))
    .orderBy(F.col("num_flights").desc())
    .limit(20)
    .toPandas()
)

fig = px.bar(
    dest_counts,
    x="num_flights",
    y="destination_airport_name",
    orientation="h",
    color="num_flights",
    color_continuous_scale="Blues",
    title="Top 20 Destinations by Flight Count (Cleaned Data)",
    text="num_flights"
)
fig.update_traces(textposition="outside")
fig.update_layout(
    template="plotly_white",
    xaxis_title="Number of Flights",
    yaxis_title="Destination",
    yaxis=dict(autorange="reversed"),
    title_font=dict(size=18, family="Arial", color="black"),
    margin=dict(l=60, r=40, t=60, b=60)
)

fig.show()

# Flights by Terminal

In [44]:
term_counts = (
    clean_df.groupBy("movement_terminal")
    .agg(F.count("*").alias("count"))
    .orderBy(F.col("count").desc())
    .toPandas()
)
fig = px.pie(
    term_counts,
    names="movement_terminal",
    values="count",
    hole=0.4,
    color_discrete_sequence=px.colors.sequential.Blues[::-1],
    title="Flights by Terminal (Cleaned Data)"
)
fig.update_traces(textinfo="percent+label")
fig.update_layout(
    template="plotly_white",
    title_font=dict(size=18, family="Arial", color="black"),
    margin=dict(l=40, r=40, t=60, b=60)
)

fig.show()